In [5]:
import numpy as np
import pandas as pd
import joblib
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [6]:
df = pd.read_csv("sample_200k.csv", low_memory=False)
df['target'] = df['loan_status'].apply(lambda x: 1 if str(x) in ["Charged Off","Default"] else 0)

In [7]:
target = 'target'
X = df.drop(columns=[target], errors='ignore')
y = df[target].values

In [8]:
drop_cols = [c for c in ['id','member_id'] if c in X.columns]
if drop_cols:
    X = X.drop(columns=drop_cols)

In [9]:
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()

In [11]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


In [12]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor_new = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, cat_cols)
], remainder='drop')

In [13]:
print("Fitting new preprocessor on full dataset...")
preprocessor_new.fit(X, y)
X_prep = preprocessor_new.transform(X)       
print("Transformed shape:", X_prep.shape)

joblib.dump(preprocessor_new, "final_preprocessor_fitted.pkl")
print("Saved final_preprocessor_fitted.pkl")

import numpy as np
if hasattr(X_prep, "toarray"):
    states = X_prep.toarray().astype(np.float32)
else:
    states = np.asarray(X_prep, dtype=np.float32)

Fitting new preprocessor on full dataset...
Transformed shape: (200000, 149)
Saved final_preprocessor_fitted.pkl


In [14]:

df['profit_if_paid'] = df['loan_amnt'] * df['int_rate']
def compute_reward(row):
    return -row['loan_amnt'] if row['target']==1 else row['profit_if_paid']
df['reward'] = df.apply(compute_reward, axis=1)

actions = np.ones(len(df), dtype=np.int32)
rewards = df['reward'].values.astype(np.float32)
next_states = states.copy()
dones = np.ones(len(df), dtype=np.int32)

# save RL dataset
np.savez("offline_rl_dataset.npz",
         states=states,
         actions=actions,
         rewards=rewards,
         next_states=next_states,
         dones=dones)

print("Saved offline_rl_dataset.npz")
print("All done. X_prep shape:", states.shape)

Saved offline_rl_dataset.npz
All done. X_prep shape: (200000, 149)
